# Introducción

El objetivo de este cuaderno es simplemente demostrar el uso de las librerías de manipulación de datos de RAPIDS: **cuDF** para la gestión de DataFrames, **cupy** para el manejo de datos y operaciones de transformación y cálculo con los valores de los DataFrames y **cuXfilter** para la visualización de datos en gráficas. Los puntos principales a desarrollar son:
* Lectura de datos en formato CSV y agregación en un conjunto de desarrollo (para futuros usos de tratamiento/ciencia de datos o machine learning).
* Manipulación básica de datos (operaciones simples sobre columnas).
* Manipulación compleja de datos (manipulación de cadenas de texto, transformaciones aritméticas y/o geográficas).
* Creación y visualización de mapas de datos en dos dimensiones.

# Sección 1: Carga y manipulación de datos en GPU con RAPIDS

## Carga de datos

Importaciones
* **os** y **os.path**: importan utilidades de Python para tratamiento de ficheros y comprobación de rutas.
* **cudf** y **cupy**: librerías de RAPIDS.

In [ ]:
import os
import os.path

import cudf
import cupy as cp

Ficheros de utilidades:
* **f_northing**: conversión de coordenadas latitud-longitud a norte-este, empleados en manejo de mapas y coordenadas.
* **f_northing_numpy**: equivalente de f_northing para estructuras de numpy.
* **f_static_data**: funciones para cargar listas de datos estáticos, tales como nombres de ciudades a utilizar desde el directorio /data, columnas a leer por cada fichero CSV, y columnas a usar en el entrenamiento del modelo.
* **f_utils**: utilidades de conversión de datos a tipos float32/float64, limpiado de strings de precios, factorización de columnas complejas en mapas numéricos, etc.

In [ ]:
%run ../utils/f_northing.py
%run ../utils/f_northing_numpy.py
%run ../utils/f_static_data.py
%run ../utils/f_utils.py

Inicialización de datos:
* **cities_to_use**: lista de directorios dentro de /data sobre los que leer datasets en formato CSV. Cada directorio es una ciudad, área, estado o país.
* **columns_to_use**: lista de columnas a leer dentro de cada CSV. Todas las columnas deben existir y sus nombres deben coincidir.
* **columns_to_fit**: lista de columnas a utilizar para el entrenamiento de modelos de machine learning. Nos permite hacer múltiples usos de un dataset sin necesidad de leerlo de nuevo.

Los datos han sido obtenidos del [repositorio público de AirBnB](http://insideairbnb.com/get-the-data.html), por cada ciudad se utilizan todos los datasets de listados (listings.tar.gz) disponibles para el año 2020.

In [ ]:
cities_to_use = ['sevilla']
#cities_to_use = ['shanghai']
#cities_to_use = cities_to_use_1()
#cities_to_use = cities_to_use_2()

columns_to_use = ['host_id', 'host_response_rate', 'host_acceptance_rate', 'latitude', 'longitude', 
                  'accommodates', 'price', 'number_of_reviews', 'reviews_per_month', 'neighbourhood_cleansed']

Debido a la cantidad de diferentes ficheros que debemos leer, y a que los diferentes _scraping_ realizados por el equipo(s) de AirBnB no siempre son iguales, podemos encontrar que en el dataset de algunos meses para algunas ciudades faltan columnas.

La siguiente celda recorre las ciudades disponibles y comprueba en qué datasets faltan columnas, dado que las trazas de error de RAPIDS y pandas no siempre son detalladas. El resultado esperado es que no se imprima ninguna línea por la consola, ya que significaría que todos los datasets disponibles tienen las columnas necesarias.

Esta celda está desactivada por defecto y es opcional, sólo requerida si se desea comprobar el estado de uno o más ficheros previa ejecución del paso de agregación, o si dicho paso no se ejecuta correctamente.

El DataFrame **listings** representa el conjunto de datos final sobre el que realizar operaciones de ciencia de datos o machine learning. Por cada directorio de datos, leemos todos los ficheros CSV disponibles y los agregamos a **listings**.
* **standard_object_type**: algunas columnas no tienen un formato único de datos; esto puede deberse a diferentes departamentos o metodologías empleadas durante los varios *scraping* de datos de la fuente. Esta función convierte columnas (detectadas manualmente a priori) en tipo genérico *object*, que después convertiremos a un tipo de datos más apropiado para nuestro uso.
* **drop_duplicates**: dado que utilizamos múltiples *datasets* por ciudades, hay listados de AirBnB que no cambian durante varios meses. En este caso, no necesitamos datos redundantes, y los removemos del DataFrame final.
* **reset_index**: los datos de cada fichero se han unido mediante un DataFrame diferente, y por lo tanto los índices del DataFrame quedan mezclados al final del proceso. Con esta instrucción forzamos el índice de **listings** a ordenarse. La instrución añade una columna adicional con el índice antiguo, que podemos obviar con el parámetro drop=**True**.
* **shape**: instrucción que devuelve el tamaño completo del DataFrame en formato (filas, columnas)

In [ ]:
%%time
listings = cudf.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df = cudf.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if listings.size == 0:
                    listings = temp_df
                else:
                    for column in listings.columns:
                        if listings[column].dtype != temp_df[column].dtype:
                            print('Found error: '+column+' type '+listings[column].dtype.name+' doesnt match '+temp_df[column].dtype.name)
                    listings = listings.append(temp_df)
                    
listings = listings.drop_duplicates().reset_index(drop=True)
listings.shape

La instrucción nvidia-smi nos permite leer la información de uso de la(s) GPU(s) disponible(s). En particular, es útil para controlar cuánta memoria de vídeo (VRAM) está en uso. Dado que RAPIDS carga todos los datos en la GPU para optimizar el acceso a la información, hay que controlar que haya suficiente memoria antes de leer datos.

Adicionalmente, algunas técnicas de machine learning requieren espacio libre igual al del *dataset* en uso, ya que lo duplican por completo. Por lo tanto, es aconsejable tener disponible al menos el doble de la memoria empleada por el *dataset* en uso.

In [ ]:
!nvidia-smi

## Tratamiento de datos con cuDF y cuPY

Tratamiento simple:
* **type_conversion**: convertimos todas las columnas a un tipo de datos común: *float32* y *float64* son los más aceptados por los algoritmos de RAPIDS.
* **column_factorize**: dado que algunos algoritmos no suportan datos no numéricos, factorizamos las columnas de texto. La operación factorize() convierte una columna con valores no numéricos a un mapa en el que cada valor único se representa por un integer. En nuestro caso, no necesitaremos el mapa que reconoce cada valor, pero es posible usarlo para devolver formato a los datos de cara a estudio o representación.

In [ ]:
%time
type_conversion(listings, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month'])
column_factorize(listings, ['neighbourhood_cleansed'])

Tratamiento de cadenas:
* **clean_format_strings**: eliminamos el carácter '%' de campos que lo contienen, y convertimos el valor resultante a *float32*.
* **clean_format_price**: similar a la función anterior, pero eliminando caracteres de moneda (en nuestro caso, '$' y la coma de cantidades numéricas).

In [ ]:
%%time
clean_format_strings(listings, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price(listings, ['price'])

## Generación de datos geográficos

A fin de visualizar los datos en un formato 2D similar a un mapa, convertimos las coordatas latitud-longitud en distancias norte-este, y les asignamos columnas nuevas.

In [ ]:
%%time
cupy_lat = cp.asarray(listings['latitude'])
cupy_long = cp.asarray(listings['longitude'])
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
listings['northing'] = cudf.Series(n_cupy_array).astype('float32')
listings['easting'] = cudf.Series(e_cupy_array).astype('float32')

Resultado final del tratamiento de datos:
* dtypes: tipo de datos de cada columna.
* head: visuzalización de las primeras 5 filas del DataFrame.

In [ ]:
listings.dtypes

In [ ]:
listings.head()

La visualización de datos se realizará en la sección 3.

# Sección 2: Carga y manipulación de datos en CPU con pandas y numPy

## Carga de datos

Importaciones
* pandas y numpy: librerías de manejo de DataFrames y gestión numérica, de vectores y tablas. Equivalentes a cudf y cupy respectivamente para CPU.

In [ ]:
import pandas as pd
import numpy as np

La lectura y tratamiento de los datos iniciales sigue el mismo proceso que para GPU:
1. Creamos un nuevo DataFrame vacío.
2. Recorremos los directorios disponibles.
3. Agregamos los CSV, convirtiendo columnas dispares a tipo *object* si fuera necesario.
4. Eliminamos duplicados.
5. Reordenamos el índice.

In [ ]:
%%time
listings_cpu = pd.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df_cpu = pd.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df_cpu, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if listings_cpu.size == 0:
                    listings_cpu = temp_df_cpu
                else:
                    for column in listings_cpu.columns:
                        if listings_cpu[column].dtype != temp_df_cpu[column].dtype:
                            print('Found error: '+column+' type '+listings_cpu[column].dtype.name+' doesnt match '+temp_df_cpu[column].dtype.name)
                    listings_cpu = listings_cpu.append(temp_df_cpu)
                    
listings_cpu = listings_cpu.drop_duplicates().reset_index(drop=True)
listings_cpu.shape

## Tratamiento de datos

Las funciones **type_conversion**, **column_factorize** y **clean_format_strings** son las mismas empleadas para GPU, ya que las operaciones subyacentes tienen la misma semántica y uso para cuDF/cupy como para pandas/numpy.

* **clean_format_price_cpu**: versión específica del tratado de cadenas de precios para CPU. Es necesario cambiar ligeramente el órden y número de operaciones para que numpy pueda ejecutar el mismo reemplazo de cadenas de precios que en GPU.

In [ ]:
%time
type_conversion(listings_cpu, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month'])
column_factorize(listings_cpu, ['neighbourhood_cleansed'])

In [ ]:
%%time
clean_format_strings(listings_cpu, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price_cpu(listings_cpu, ['price'])

## Generación de datos geográficos

La única diferencia en la conversión de coordenadas latitud-logitud a norte-este es que utilizamos funciones aritméticas provistas por numpy en lugar de cupy.

In [ ]:
%%time
numpy_lat = listings_cpu['latitude'].to_numpy()
numpy_long = listings_cpu['longitude'].to_numpy()
n_numpy_array, e_numpy_array = latlong2osgbgrid_numpy(numpy_lat, numpy_long)
listings_cpu['northing'] = pd.Series(n_numpy_array).astype('float32')
listings_cpu['easting'] = pd.Series(e_numpy_array).astype('float32')

In [ ]:
listings_cpu.dtypes

In [ ]:
listings_cpu.head()

El comando %reset nos permite limpiar todas las variables sin reiniciar el kernel. Esto nos permite aprovechar al máximo la gestión de memoria inicializada por RAPIDS, ejecutando pruebas múltiples veces en el mismo kernel inicializado.

In [ ]:
%reset -f

# Sección 3: Visualización de resultados mediante cuXfilter

La librería cuXfilter nos permite visualizar datos gráficamente. Vamos a visualizar los listados de AirBnB en las ciudades seleccionadas durante el año 2020, con un selector por distritos/barrios.

Creamos un gráfico de tipo *scatter* para visualizar puntos en una escala 2D (en este caso marcada por las coordenadas norte-este), así como un *widget* donde escribimos los nombres de las zonas disponibles. Podremos seleccionar cada zona y ver los listados específicos.

***Nota:*** es necesario guardar el resultado de aplicar la operación **factorize** de cuDF/pandas sobre la columna 'neighbourhood_cleansed', lo cual no está soportado por las funciones de utilidad. Si se desea ejecutar este mapa, es necesario aplicar la operación manualmente en el notebook.

Abrimos el gráfico en un *dashboard* creado en el navegador:

Para ver el gráfico sin abrir un widget podemos usar el siguiente comando:

Para detener la ejecución del gráfico, podemos usar el siguiente comando: